In [1]:
import flopy, os
import sys
import platform
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from seaborn import load_dataset
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import Workbook
import flopy.utils.binaryfile as bf

flopy is installed in C:\Users\LENOVO\miniconda3\lib\site-packages\flopy


In [2]:
# Directorio de trabajo:
model_ws= "../../Outputs/Resultados_Base/"
modelname= 'RUCM_Base'
exe_name= '../../Solver/mf2005.exe'
wb_name = 'WaterBudget_Base'
ml= flopy.modflow.Modflow.load(modelname+'.nam', exe_name=exe_name, 
                               model_ws=model_ws, check=False)

In [3]:
from flopy.utils.zonbud import ZoneBudget, read_zbarray
zona_total = read_zbarray(os.path.join(model_ws + '/' + modelname + '.zontotal'))
balance_total = ZoneBudget(model_ws + '/' + modelname + '.cbc', zona_total, kstpkper=None)
df_total = balance_total.get_dataframes()

# *1. BALANCE TOTAL* 

In [4]:
# Arreglo del dataframe Zonebudget:
time = ml.dis.get_totim()
nper = ml.dis.nper
entrada_t = []
salida_t = []
balance_t = []
error_t = []
discrep_t = []
for i in time:
    entrada_t.append(df_total.loc[(i,'TOTAL_IN'), ['ZONE_1']])
    salida_t.append(df_total.loc[(i,'TOTAL_OUT'), ['ZONE_1']])
    error_t.append(df_total.loc[(i,'PERCENT_DISCREPANCY'), ['ZONE_1']])
    
for i in range(len(entrada_t)):
    balance_t.append(entrada_t[i]-salida_t[i])
    discrep_t.append((entrada_t[i]-salida_t[i])/entrada_t[i]*100)

arrays_inout_t = [time, np.array(['IN-OUT']*nper)]
df1_t = pd.DataFrame(balance_t, index=arrays_inout_t)

arrays_discrep_t = [time, np.array(['PERCENT_DISCREPANCY']*nper)]
df2_t = pd.DataFrame(discrep_t, index=arrays_discrep_t)

# Reemplazando en df:
for i in time:
    df_total.loc[(i,'IN-OUT'),'ZONE_1'] = df1_t.loc[(i,'IN-OUT'),'ZONE_1']

for i in time:
    df_total.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_1'] = df2_t.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_1']

df_total = df_total.loc[(time,['FROM_STORAGE', 'FROM_CONSTANT_HEAD', 'FROM_WELLS', 'FROM_RIVER_LEAKAGE',
                                  'FROM_HEAD_DEP_BOUNDS', 'FROM_RECHARGE', 'TOTAL_IN', 'TO_STORAGE', 'TO_CONSTANT_HEAD',
                                  'TO_WELLS', 'TO_RIVER_LEAKAGE', 'TO_HEAD_DEP_BOUNDS', 'TO_RECHARGE', 'TOTAL_OUT', 
                                  'IN-OUT', 'PERCENT_DISCREPANCY']), ['ZONE_1']]

In [5]:
#df_total.to_csv(model_ws + '/' + wb_name +'.csv')

### *1.1. BALANCE TOTAL (m3/d)* 

In [6]:
def f(x):
    df_1 = df_total.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    return df_3

perlen = []
for i in range(nper):
    perlen.append(ml.dis.perlen[i])
Periodo = [i+1 for i in range(nper)]

In [7]:
lista = list(zip(Periodo, perlen, time, f('FROM_STORAGE'), f('FROM_CONSTANT_HEAD'),f('FROM_WELLS'),
                 f('FROM_RIVER_LEAKAGE'),f('FROM_HEAD_DEP_BOUNDS'), f('FROM_RECHARGE'), 
                 f('TOTAL_IN'), f('TO_STORAGE'), f('TO_CONSTANT_HEAD'), f('TO_WELLS'), 
                 f('TO_RIVER_LEAKAGE'), f('TO_HEAD_DEP_BOUNDS'), f('TO_RECHARGE'),
                 f('TOTAL_OUT'), f('IN-OUT'), f('PERCENT_DISCREPANCY')))
ship_columns = ['Stress Period', 'Time','Total Time','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 
                'TOTAL IN (m3/day)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE',
                'TOTAL OUT (m3/day)', 'IN-OUT (m3/day)', 'PERCENT DISCREPANCY']

In [8]:
Balance = pd.DataFrame(columns=ship_columns, data=lista)

### *1.2. BALANCE TOTAL (m3/mes)* 

In [9]:
def f2(x):
    df_1 = df_total.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    df_4 = []
    for i in range(len(perlen)):
                   df_4.append(df_3[i]*perlen[i])       
    return df_4

In [10]:
lista_2 = list(zip(Periodo, perlen, time, f2('FROM_STORAGE'), f2('FROM_CONSTANT_HEAD'),f2('FROM_WELLS'),
                 f2('FROM_RIVER_LEAKAGE'),f2('FROM_HEAD_DEP_BOUNDS'), f2('FROM_RECHARGE'), 
                 f2('TOTAL_IN'), f2('TO_STORAGE'), f2('TO_CONSTANT_HEAD'), f2('TO_WELLS'), 
                 f2('TO_RIVER_LEAKAGE'), f2('TO_HEAD_DEP_BOUNDS'), f2('TO_RECHARGE'),
                 f2('TOTAL_OUT'), f2('IN-OUT'), f('PERCENT_DISCREPANCY')))
ship_columns_2 = ['Stress Period', 'Time','Total Time','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 
                'TOTAL IN (m3/mes)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE',
                'TOTAL OUT (m3/mes)', 'IN-OUT (m3/mes)', 'PERCENT DISCREPANCY']

In [11]:
Balance_2 = pd.DataFrame(columns=ship_columns_2, data=lista_2)

### *1.3. BALANCE ACUMULADO (m3)* 

In [12]:
def f3(x):
    df_1 = df_total.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    df_4 = []
    for i in range(len(perlen)):
        df_4.append(df_3[i]*perlen[i])
    df_5 = []
    for i in range(len(perlen)):
        if i==0:
            df_5.append(df_4[i])
        else:
            df_5.append(df_4[i]+df_5[i-1])  
    return df_5

In [13]:
lista_3 = list(zip(Periodo, perlen, time, f3('FROM_STORAGE'), f3('FROM_CONSTANT_HEAD'),f3('FROM_WELLS'),
                 f3('FROM_RIVER_LEAKAGE'),f3('FROM_HEAD_DEP_BOUNDS'), f3('FROM_RECHARGE'), 
                 f3('TOTAL_IN'), f3('TO_STORAGE'), f3('TO_CONSTANT_HEAD'), f3('TO_WELLS'), 
                 f3('TO_RIVER_LEAKAGE'), f3('TO_HEAD_DEP_BOUNDS'), f3('TO_RECHARGE'),
                 f3('TOTAL_OUT'), f3('IN-OUT'), f('PERCENT_DISCREPANCY')))
ship_columns_3 = ['Stress Period', 'Time','Total Time','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 
                'TOTAL IN (m3)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE',
                'TOTAL OUT (m3)', 'IN-OUT (m3)', 'PERCENT DISCREPANCY']

In [14]:
Balance_3 = pd.DataFrame(columns=ship_columns_3, data=lista_3)

### *1.4. BALANCE ACUMULADO (hm3)* 

In [15]:
def f4(x):
    df_1 = df_total.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    df_4 = []
    for i in range(len(perlen)):
        df_4.append(df_3[i]*perlen[i])
    df_5 = []
    for i in range(len(perlen)):
        if i==0:
            df_5.append(df_4[i]/1000000)
        else:
            df_5.append((df_4[i]/1000000+df_5[i-1]))
    df_5 = df_5
    return df_5

In [16]:
lista_4 = list(zip(Periodo, perlen, time, f4('FROM_STORAGE'), f4('FROM_CONSTANT_HEAD'),f4('FROM_WELLS'),
                 f4('FROM_RIVER_LEAKAGE'),f4('FROM_HEAD_DEP_BOUNDS'), f4('FROM_RECHARGE'), 
                 f4('TOTAL_IN'), f4('TO_STORAGE'), f4('TO_CONSTANT_HEAD'), f4('TO_WELLS'), 
                 f4('TO_RIVER_LEAKAGE'), f4('TO_HEAD_DEP_BOUNDS'), f4('TO_RECHARGE'),
                 f4('TOTAL_OUT'), f4('IN-OUT'), f('PERCENT_DISCREPANCY')))
ship_columns_4 = ['Stress Period', 'Time','Total Time','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 
                'TOTAL IN (hm3)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE',
                'TOTAL OUT (hm3)', 'IN-OUT (hm3)', 'PERCENT DISCREPANCY']

In [17]:
Balance_4 = pd.DataFrame(columns=ship_columns_4, data=lista_4)
#Balance_4.to_csv(model_ws + '/' + wb_name +'_acum_hm3.csv')

### *1.5. BALANCE ANUAL (hm3)* 

In [18]:
def f5(x):
    df_1 = df_total.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    df_4 = []
    for i in range(len(perlen)):
        df_4.append(df_3[i]*perlen[i])
    
    df_5 = []
    v = int(len(perlen)/12) # Cada 12 meses
    for i in range(v):
        df_5.append(sum(df_4[12*i:12*(i+1)])/1000000)
    return df_5

Years = []
anio_hidro1 = 1980
anio_hidro2 = 81
for i in range(int(len(perlen)/12)):
    if anio_hidro1 < 1999:
        Years.append(str(anio_hidro1)+'/'+str(anio_hidro2))
    else:
        if anio_hidro1 < 2009:
            Years.append(str(anio_hidro1)+'/'+ str(0)+str(anio_hidro2-100))
        else:
            Years.append(str(anio_hidro1)+'/'+str(anio_hidro2-100))
       
    anio_hidro1 = anio_hidro1+1
    anio_hidro2 = anio_hidro2+1
    

In [19]:
lista_5 = list(zip(Years, f5('FROM_STORAGE'), f5('FROM_CONSTANT_HEAD'),f5('FROM_WELLS'),
                 f5('FROM_RIVER_LEAKAGE'),f5('FROM_HEAD_DEP_BOUNDS'), f5('FROM_RECHARGE'), 
                 f5('TOTAL_IN'), f5('TO_STORAGE'), f5('TO_CONSTANT_HEAD'), f5('TO_WELLS'), 
                 f5('TO_RIVER_LEAKAGE'), f5('TO_HEAD_DEP_BOUNDS'), f5('TO_RECHARGE'),
                 f5('TOTAL_OUT'), f5('IN-OUT'), f5('PERCENT_DISCREPANCY')))
ship_columns_5 = ['Year','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 
                'TOTAL IN (hm3/year)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE',
                'TOTAL OUT (hm3/year)', 'IN-OUT (hm3/year)', 'PERCENT DISCREPANCY']

In [20]:
Balance_5 = pd.DataFrame(columns=ship_columns_5, data=lista_5)

### *1.6. BALANCE PROMEDIO ANUAL (hm3)* 

In [21]:
# Funcion para los promedios anuales:
def f_wb_prom(entrada,salida):
    wb_entrada = f5(entrada)
    wb_salida = f5(salida)
    wb_entrada_prom = sum(wb_entrada)/len(wb_entrada)
    wb_salida_prom = sum(wb_salida)/len(wb_salida)
    wb_delta_prom = wb_entrada_prom - wb_salida_prom

    return [wb_entrada_prom, wb_salida_prom, wb_delta_prom]

In [22]:
wb_prom = [f_wb_prom('FROM_RECHARGE','TO_RECHARGE'), f_wb_prom('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS'),
           f_wb_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE'), f_wb_prom('FROM_WELLS','TO_WELLS')]

In [23]:
entrada_prom = []
for i in range(len(wb_prom)):
    entrada_prom.append(round((wb_prom[i][0]),2))
entrada_prom.append(sum(entrada_prom))

salida_prom = []
for i in range(len(wb_prom)):
    salida_prom.append(round((wb_prom[i][1]),2))
salida_prom.append(sum(salida_prom))

delta_prom = []
for i in range(len(wb_prom)):
    delta_prom.append(round((wb_prom[i][2]),2))
delta_prom.append(sum(delta_prom))

In [24]:
descr_prom = ['Recarga', 'Otras Masas', 'Río - Acuífero', 
             'Bombeo', 'TOTAL']
lista_prom = list(zip(descr_prom,entrada_prom, salida_prom, delta_prom))
ship_columns_prom = ['Descripción','Entrada (hm3)','Salida (hm3)','Delta (hm3)']

In [25]:
WB_resumen = pd.DataFrame(columns=ship_columns_prom, data=lista_prom)
#WB_resumen.to_csv(model_ws + '/' + wb_name +'_PromedioAnual.csv')
WB_resumen

,Descripción,Entrada (hm3),Salida (hm3),Delta (hm3)
0,Recarga,53.92,0.00,53.92
1,Otras Masas,15.62,43.04,-27.42
2,Río - Acuífero,1.87,11.16,-9.28
3,Bombeo,0.00,9.52,-9.52
4,TOTAL,71.41,63.72,7.70


# *2. BALANCE POR MASAS SUBTERRANEAS (hm3)* 

In [26]:
zona_masas = read_zbarray(os.path.join(model_ws + '/' + modelname + '.zon_masas'))
balance_masas = ZoneBudget(model_ws + '/' + modelname + '.cbc', zona_masas, kstpkper=None)
df_masas = balance_masas.get_dataframes()

### *2.1. BALANCE DE MASAS TOTAL (m3/d)* 

In [27]:
# Arreglo del dataframe Zonebudget:
entrada_m = []
salida_m = []
balance_m = []
error_m = []
discrep_m = []
for i in time:
    entrada_m.append(df_masas.loc[(i,'TOTAL_IN'), ['ZONE_1', 'ZONE_2']])
    salida_m.append(df_masas.loc[(i,'TOTAL_OUT'), ['ZONE_1', 'ZONE_2']])
    error_m.append(df_masas.loc[(i,'PERCENT_DISCREPANCY'), ['ZONE_1', 'ZONE_2']])
    
for i in range(len(entrada_m)):
    balance_m.append(entrada_m[i]-salida_m[i])
    discrep_m.append((entrada_m[i]-salida_m[i])/entrada_m[i]*100)
    
arrays_inout_m = [time, np.array(['IN-OUT']*nper)]
df1_m = pd.DataFrame(balance_m, index=arrays_inout_m)

arrays_discrep_m = [time, np.array(['PERCENT_DISCREPANCY']*nper)]
df2_m = pd.DataFrame(discrep_m, index=arrays_discrep_m)

# Reemplazando en df:
for i in time:
    df_masas.loc[(i,'IN-OUT'),['ZONE_1','ZONE_2']] = df1_m.loc[(i,'IN-OUT'),['ZONE_1','ZONE_2']]
    
for i in time:
    df_masas.loc[(i,'PERCENT_DISCREPANCY'),['ZONE_1','ZONE_2']] = df2_m.loc[(i,'PERCENT_DISCREPANCY'),['ZONE_1','ZONE_2']]
    
df_masas = df_masas.loc[(time,['FROM_STORAGE', 'FROM_CONSTANT_HEAD', 'FROM_WELLS','FROM_RIVER_LEAKAGE',
                                'FROM_HEAD_DEP_BOUNDS', 'FROM_RECHARGE','FROM_ZONE_1',
                                'FROM_ZONE_2', 'TOTAL_IN', 'TO_STORAGE', 'TO_CONSTANT_HEAD', 'TO_WELLS',
                                'TO_RIVER_LEAKAGE', 'TO_HEAD_DEP_BOUNDS', 'TO_RECHARGE',
                                'TO_ZONE_1', 'TO_ZONE_2', 'TOTAL_OUT', 'IN-OUT',
                                'PERCENT_DISCREPANCY']), ['ZONE_1','ZONE_2']]

In [28]:
def fm(x):
    df_1 = df_masas.loc[(time,[x]), :]
    df_2 = df_1.values.tolist()
    df_3 = [y for x in df_2 for y in x]
    return df_3

In [29]:
perlen_m = []
for i in range(nper):
    perlen_m.append(ml.dis.perlen[i])
    perlen_m.append(ml.dis.perlen[i])
    
Periodo_m = []
for i in range(nper):
    Periodo_m.append(i+1)
    Periodo_m.append(i+1)

time_m = []
for i in range(nper):
    time_m.append(time[i])
    time_m.append(time[i])

list_zone = []
for i in range(nper):
    list_zone.append('ZONE 1')
    list_zone.append('ZONE 2')

In [30]:
lista_m1 = list(zip(Periodo_m, perlen_m, time_m, list_zone, fm('FROM_STORAGE'), fm('FROM_CONSTANT_HEAD'),
                    fm('FROM_WELLS'), fm('FROM_RIVER_LEAKAGE'),fm('FROM_HEAD_DEP_BOUNDS'), 
                    fm('FROM_RECHARGE'),fm('FROM_ZONE_1'), fm('FROM_ZONE_2'),
                    fm('TOTAL_IN'), fm('TO_STORAGE'), fm('TO_CONSTANT_HEAD'), fm('TO_WELLS'),
                    fm('TO_RIVER_LEAKAGE'), fm('TO_HEAD_DEP_BOUNDS'), fm('TO_RECHARGE'),
                    fm('TO_ZONE_1'), fm('TO_ZONE_2'),fm('TOTAL_OUT'),
                    fm('IN-OUT'), fm('PERCENT_DISCREPANCY')))
ship_columns_m1 = ['Stress Period', 'Time','Total Time','ZONES','FROM STORAGE', 'FROM CONSTANT HEAD',
                   'FROM WELLS', 'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE',
                   'FROM ZONE 1', 'FROM ZONE 2', 'TOTAL IN (m3/day)', 'TO STORAGE',
                   'TO CONSTANT HEAD', 'TO WELLS', 'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS',
                   'TO RECHARGE', 'TO ZONE 1', 'TO ZONE 2', 'TOTAL OUT (m3/day)',
                   'IN-OUT (m3/day)', 'PERCENT DISCREPANCY']

In [31]:
Balance_masas = pd.DataFrame(columns=ship_columns_m1, data=lista_m1)

### *2.2. BALANCE DE MASAS ANUAL (hm3)* 

In [32]:
def fm_anual(x, zone):
    df_1 = df_masas.loc[(time,[x]), zone]
    df_2 = df_1.values.tolist()
    df_4 = []
    for i in range(len(perlen)):
        df_4.append(df_2[i]*perlen[i])
    
    df_5 = []
    v = int(len(perlen)/12) # Cada 12 meses
    for i in range(v):
        df_5.append(sum(df_4[12*i:12*(i+1)])/1000000)
    return df_5

Years = []
anio_hidro1 = 1980
anio_hidro2 = 81
for i in range(int(len(perlen)/12)):
    if anio_hidro1 < 1999:
        Years.append(str(anio_hidro1)+'/'+str(anio_hidro2))
    else:
        if anio_hidro1 < 2009:
            Years.append(str(anio_hidro1)+'/'+ str(0)+str(anio_hidro2-100))
        else:
            Years.append(str(anio_hidro1)+'/'+str(anio_hidro2-100))
       
    anio_hidro1 = anio_hidro1 +1
    anio_hidro2 = anio_hidro2+1

#### *2.2.1. BALANCE DE MASAS ANUAL - RU (hm3)* 

In [33]:
masa_RU =['Requena-Utiel']*int(len(perlen)/12)

In [34]:
lista_RU = list(zip(Years, masa_RU, fm_anual('FROM_STORAGE','ZONE_1'), fm_anual('FROM_CONSTANT_HEAD','ZONE_1'),
                    fm_anual('FROM_WELLS','ZONE_1'), fm_anual('FROM_RIVER_LEAKAGE','ZONE_1'),fm_anual('FROM_HEAD_DEP_BOUNDS','ZONE_1'),
                    fm_anual('FROM_RECHARGE','ZONE_1'),fm_anual('FROM_ZONE_2','ZONE_1'),fm_anual('TOTAL_IN','ZONE_1'),
                    fm_anual('TO_STORAGE','ZONE_1'), fm_anual('TO_CONSTANT_HEAD','ZONE_1'), fm_anual('TO_WELLS','ZONE_1'),
                    fm_anual('TO_RIVER_LEAKAGE','ZONE_1'), fm_anual('TO_HEAD_DEP_BOUNDS','ZONE_1'), fm_anual('TO_RECHARGE','ZONE_1'),
                    fm_anual('TO_ZONE_2','ZONE_1'), fm_anual('TOTAL_OUT','ZONE_1'), fm_anual('IN-OUT','ZONE_1'),
                    fm_anual('PERCENT_DISCREPANCY','ZONE_1')))
ship_columns_RU = ['Year', 'Masa','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 'FROM ZONE 2',
                'TOTAL IN (hm3/year)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE', 'TO ZONE 2',
                'TOTAL OUT (hm3/year)', 'IN-OUT (hm3/year)', 'PERCENT DISCREPANCY']

In [35]:
Balance_RU = pd.DataFrame(columns=ship_columns_RU, data=lista_RU)
Balance_RU.head()

,Year,Masa,FROM STORAGE,FROM CONSTANT HEAD,FROM WELLS,FROM RIVER LEAKAGE,FROM HEAD DEP BOUNDS,FROM RECHARGE,FROM ZONE 2,TOTAL IN (hm3/year),TO STORAGE,TO CONSTANT HEAD,TO WELLS,TO RIVER LEAKAGE,TO HEAD DEP BOUNDS,TO RECHARGE,TO ZONE 2,TOTAL OUT (hm3/year),IN-OUT (hm3/year),PERCENT DISCREPANCY
0,1980/81,Requena-Utiel,539.761326,0.0,0.0,1.766334,25.841008,40.910135,7.473885,615.752697,578.382898,0.0,4.551362,5.919061,8.424032,0.0,18.444112,615.721467,0.031231,2.854410e-06
1,1981/82,Requena-Utiel,232.971302,0.0,0.0,1.757906,20.795016,5.847525,7.653872,269.025622,234.629647,0.0,4.531660,5.489209,8.418451,0.0,15.958552,269.027526,-0.001903,-2.969765e-07
2,1982/83,Requena-Utiel,174.702307,0.0,0.0,1.717645,18.950447,48.772413,7.928815,252.071630,218.975997,0.0,4.511957,5.147557,8.404569,0.0,15.015658,252.055738,0.015892,2.905027e-06
3,1983/84,Requena-Utiel,140.873367,0.0,0.0,1.791863,18.243311,14.653503,8.091437,183.653479,151.203446,0.0,4.492254,4.982999,8.411259,0.0,14.543025,183.632984,0.020495,3.066801e-06
4,1984/85,Requena-Utiel,123.752756,0.0,0.0,1.777369,17.402038,49.779313,8.403425,201.114903,169.478784,0.0,4.474383,4.830652,8.377829,0.0,13.943565,201.105213,0.009690,4.814379e-06


#### *2.2.2. BALANCE DE MASAS ANUAL - CM (hm3)* 

In [36]:
masa_CM =['Cabrillas-Malacara']*int(len(perlen)/12)

In [37]:
lista_CM = list(zip(Years, masa_CM, fm_anual('FROM_STORAGE','ZONE_2'), fm_anual('FROM_CONSTANT_HEAD','ZONE_2'),
                    fm_anual('FROM_WELLS','ZONE_2'), fm_anual('FROM_RIVER_LEAKAGE','ZONE_2'),fm_anual('FROM_HEAD_DEP_BOUNDS','ZONE_2'),
                    fm_anual('FROM_RECHARGE','ZONE_2'),fm_anual('FROM_ZONE_1','ZONE_2'),fm_anual('TOTAL_IN','ZONE_2'),
                    fm_anual('TO_STORAGE','ZONE_2'), fm_anual('TO_CONSTANT_HEAD','ZONE_2'), fm_anual('TO_WELLS','ZONE_2'),
                    fm_anual('TO_RIVER_LEAKAGE','ZONE_2'), fm_anual('TO_HEAD_DEP_BOUNDS','ZONE_2'), fm_anual('TO_RECHARGE','ZONE_2'),
                    fm_anual('TO_ZONE_1','ZONE_2'), fm_anual('TOTAL_OUT','ZONE_2'), fm_anual('IN-OUT','ZONE_2'),
                    fm_anual('PERCENT_DISCREPANCY','ZONE_2')))
ship_columns_CM = ['Year', 'Masa','FROM STORAGE', 'FROM CONSTANT HEAD', 'FROM WELLS',
                'FROM RIVER LEAKAGE', 'FROM HEAD DEP BOUNDS', 'FROM RECHARGE', 'FROM ZONE 1',
                'TOTAL IN (hm3/year)', 'TO STORAGE', 'TO CONSTANT HEAD', 'TO WELLS', 
                'TO RIVER LEAKAGE', 'TO HEAD DEP BOUNDS', 'TO RECHARGE', 'TO ZONE 1',
                'TOTAL OUT (hm3/year)', 'IN-OUT (hm3/year)', 'PERCENT DISCREPANCY']

In [38]:
Balance_CM = pd.DataFrame(columns=ship_columns_CM, data=lista_CM)

### *2.3. BALANCE DE MASAS - PROMEDIO ANUAL (hm3)* 

In [39]:
# Funcion para los promedios anuales:
def f_masa_prom(entrada,salida,zone):
    masa_entrada = fm_anual(entrada,zone)
    masa_salida = fm_anual(salida,zone)
    masa_entrada_prom = sum(masa_entrada)/len(masa_entrada)
    masa_salida_prom = sum(masa_salida)/len(masa_salida)
    masa_delta_prom = masa_entrada_prom - masa_salida_prom

    return [masa_entrada_prom, masa_salida_prom, masa_delta_prom]

#### *2.3.1. RU - PROMEDIO ANUAL (hm3)* 

In [40]:
RU_prom = [f_masa_prom('FROM_RECHARGE','TO_RECHARGE','ZONE_1'),
           f_masa_prom('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_1'),
           f_masa_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_1'),
           f_masa_prom('FROM_WELLS','TO_WELLS','ZONE_1')]

In [41]:
entrada_RU_prom = []
for i in range(len(RU_prom)):
    entrada_RU_prom.append(round((RU_prom[i][0]),2))
entrada_RU_prom.append(sum(entrada_RU_prom))

salida_RU_prom = []
for i in range(len(RU_prom)):
    salida_RU_prom.append(round((RU_prom[i][1]),2))
salida_RU_prom.append(sum(salida_RU_prom))

delta_RU_prom = []
for i in range(len(RU_prom)):
    delta_RU_prom.append(round((RU_prom[i][2]),2))
delta_RU_prom.append(sum(delta_RU_prom))

In [42]:
descr_RU_prom = ['Recarga', 'Otras Masas', 'Río - Acuífero', 
             'Bombeo', 'TOTAL']
lista_RU_prom = list(zip(descr_RU_prom,entrada_RU_prom, salida_RU_prom, delta_RU_prom))
ship_columns_RU_prom = ['Descripción','Entrada (hm3)','Salida (hm3)','Delta (hm3)']

In [43]:
RU_resumen = pd.DataFrame(columns=ship_columns_RU_prom, data=lista_RU_prom)

#### *2.3.2. CM - PROMEDIO ANUAL (hm3)* 

In [44]:
CM_prom = [f_masa_prom('FROM_RECHARGE','TO_RECHARGE','ZONE_2'),
           f_masa_prom('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_2'),
           f_masa_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_2'),
           f_masa_prom('FROM_WELLS','TO_WELLS','ZONE_2')]

In [45]:
entrada_CM_prom = []
for i in range(len(CM_prom)):
    entrada_CM_prom.append(round((CM_prom[i][0]),2))
entrada_CM_prom.append(sum(entrada_CM_prom))

salida_CM_prom = []
for i in range(len(CM_prom)):
    salida_CM_prom.append(round((CM_prom[i][1]),2))
salida_CM_prom.append(sum(salida_CM_prom))

delta_CM_prom = []
for i in range(len(CM_prom)):
    delta_CM_prom.append(round((CM_prom[i][2]),2))
delta_CM_prom.append(sum(delta_CM_prom))

In [46]:
descr_CM_prom = ['Recarga', 'Otras Masas', 'Río - Acuífero', 
             'Bombeo', 'TOTAL']
lista_CM_prom = list(zip(descr_CM_prom,entrada_CM_prom, salida_CM_prom, delta_CM_prom))
ship_columns_CM_prom = ['Descripción','Entrada (hm3)','Salida (hm3)','Delta (hm3)']

In [47]:
CM_resumen = pd.DataFrame(columns=ship_columns_CM_prom, data=lista_CM_prom)

## BALANCE TOTAL - Exportando a Excel

In [48]:
writer_WB = pd.ExcelWriter(model_ws + '/' + wb_name +'_Total.xlsx')
WB_resumen.to_excel(writer_WB,'PromedioAnual',index=True)
Balance.to_excel(writer_WB,'m3-d',index=True)
Balance_2.to_excel(writer_WB,'m3-mes',index=True)
Balance_3.to_excel(writer_WB,'acum_m3',index=True)
Balance_4.to_excel(writer_WB,'acum_hm3',index=True)
Balance_5.to_excel(writer_WB,'hm3-year',index=True)
writer_WB.save()

## BALANCE DE MASAS - Exportando a Excel

In [49]:
writer_BM = pd.ExcelWriter(model_ws + '/' + wb_name +'_Masas.xlsx')
Balance_masas.to_excel(writer_BM,'masas_m3-d',index=True)
RU_resumen.to_excel(writer_BM,'RU_PromedioAnual',index=True)
CM_resumen.to_excel(writer_BM,'CM_PromedioAnual',index=True)
Balance_RU.to_excel(writer_BM,'RU_hm3-year',index=True)
Balance_CM.to_excel(writer_BM,'CM_hm3-year',index=True)
writer_BM.save()